In [1]:
import matplotlib.pyplot as plt
import admix
import numpy as np
from admix.plot import compare_pval
from os.path import join
import pandas as pd
import math
import sys
import argparse
import openpyxl
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import copy
import statsmodels
import time
import dask.array as da
import qtl
from qtl.plot import qqplot
import matplotlib.image as mpimg
from scipy import stats 
import re
import structlog

logger = structlog.get_logger()

In [2]:
#first format the phenotype data
#use the way fits your dataset
#example here uses UKBB

trait = "250.1"

df_dir = "/u/project/sgss/UKBB/PRS-RESEARCH/03-compile-pheno/out/"
df_pheno = pd.read_csv(join(df_dir,f"{trait}.tsv"), delim_whitespace=True, low_memory=False,)
df_pheno.index = df_pheno.FID.astype(str) + "_" + df_pheno.IID.astype(str)
df_pheno = df_pheno.drop(columns=["FID", "IID"])
df_covar = pd.read_csv(join(df_dir, "covar.tsv"),delim_whitespace=True,low_memory=False)
df_covar.index = df_covar.FID.astype(str) + "_" + df_covar.IID.astype(str)
df_covar = df_covar.drop(columns=["FID", "IID"])
df_combined = pd.concat([df_pheno, df_covar], axis=1)

df_combined = df_combined.drop(columns=["DEPRIVATION_INDEX", "PC11","PC12","PC13","PC14","PC15","PC16"])

compression_opts = dict(method='zip',archive_name=f'test_combine.csv')  
df_combined.to_csv(f"/u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip", index=True, index_label="id",compression=compression_opts) 




In [32]:
df_combined

,PHENO,AGE,SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
1923142_1923142,0.0,41,0,-11.4615,3.24986,-1.566560,0.333621,-5.31024,-0.218368,1.508250,-2.161980,1.098610,1.118510
5896937_5896937,0.0,53,0,-12.5305,2.00621,-5.478720,5.895930,7.63028,0.875149,-2.023640,1.442030,1.102370,3.549290
2847894_2847894,NaN,58,1,-14.5264,1.76954,0.127117,-0.244096,-4.25753,-0.494518,0.235786,3.466990,-0.622341,1.860490
5764431_5764431,0.0,60,0,-10.8526,2.72869,-1.060580,-0.866161,-7.23222,0.021669,1.943020,-1.808820,-0.505936,0.342399
4695056_4695056,0.0,65,0,-15.4245,2.42109,-4.002410,2.316300,-0.92956,0.494015,3.379330,-5.852300,-3.074070,1.981010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738092_5738092,0.0,45,0,64.8253,-118.29300,-44.576000,-1.784150,2.40612,-10.122300,0.850589,1.359010,1.629030,-0.606739
2284715_2284715,0.0,48,1,-11.1320,5.80997,2.922140,-0.209177,-11.35140,2.627340,2.538100,0.560629,1.910830,-3.200550
3369445_3369445,0.0,51,0,-11.6389,2.01891,-2.900290,3.003380,13.27890,3.218520,0.960754,-0.290488,-1.867840,-2.128400
4700415_4700415,0.0,53,1,-12.6424,2.43072,0.178078,6.263680,16.49210,2.054440,-0.825126,2.020530,1.051790,2.304730


In [6]:
# run the script to 


for i in range(1,23):
  
    os.system(f"python computeChrom.py \
                -o /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D.{i} \
                -p /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip \
                -r /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr{i}.msp.tsv")
    
    
    
    

2023-10-25 12:41:56 [info     ] Currently working on /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr1.msp.tsv
2023-10-25 12:41:57 [info     ] Reading phenotypes and covariates in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip with shape (488266, 13)
First 5 lines of data 
                 PHENO  AGE  SEX  ...      PC8       PC9      PC10
id                                ...                             
1923142_1923142    0.0   41    0  ... -2.16198  1.098610  1.118510
5896937_5896937    0.0   53    0  ...  1.44203  1.102370  3.549290
2847894_2847894    NaN   58    1  ...  3.46699 -0.622341  1.860490
5764431_5764431    0.0   60    0  ... -1.80882 -0.505936  0.342399
4695056_4695056    0.0   65    0  ... -5.85230 -3.074070  1.981010

[5 rows x 13 columns]
2023-10-25 12:42:00 [info     ] Found mismatched phenotype and lanc. Removed
2023-10-25 12:42:00 [info     ] Found nan phenotypes. Removed


admix.assoc.marginal: 100%|██████████| 1/1 [00:04<00:00,  4.96s/it]


2023-10-25 12:42:25 [info     ] Currently working on /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr2.msp.tsv
2023-10-25 12:42:26 [info     ] Reading phenotypes and covariates in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip with shape (488266, 13)
First 5 lines of data 
                 PHENO  AGE  SEX  ...      PC8       PC9      PC10
id                                ...                             
1923142_1923142    0.0   41    0  ... -2.16198  1.098610  1.118510
5896937_5896937    0.0   53    0  ...  1.44203  1.102370  3.549290
2847894_2847894    NaN   58    1  ...  3.46699 -0.622341  1.860490
5764431_5764431    0.0   60    0  ... -1.80882 -0.505936  0.342399
4695056_4695056    0.0   65    0  ... -5.85230 -3.074070  1.981010

[5 rows x 13 columns]
2023-10-25 12:42:29 [info     ] Found mismatched phenotype and lanc. Removed
2023-10-25 12:42:29 [info     ] Found nan phenotypes. Removed


admix.assoc.marginal: 100%|██████████| 1/1 [00:04<00:00,  4.35s/it]


2023-10-25 12:42:51 [info     ] Currently computing chromosome /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr3.msp.tsv
2023-10-25 12:42:52 [info     ] Reading phenotypes and covariates in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip with shape (488266, 13)
First 5 lines of data 
                 PHENO  AGE  SEX  ...      PC8       PC9      PC10
id                                ...                             
1923142_1923142    0.0   41    0  ... -2.16198  1.098610  1.118510
5896937_5896937    0.0   53    0  ...  1.44203  1.102370  3.549290
2847894_2847894    NaN   58    1  ...  3.46699 -0.622341  1.860490
5764431_5764431    0.0   60    0  ... -1.80882 -0.505936  0.342399
4695056_4695056    0.0   65    0  ... -5.85230 -3.074070  1.981010

[5 rows x 13 columns]
2023-10-25 12:42:55 [info     ] Found mismatched phenotype and lanc. Removed
2023-10-25 12:42:55 [info     ] Found nan phenotypes. Removed


...

2023-10-25 12:49:08 [info     ] Saving to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D.20.csv.gz
2023-10-25 12:49:14 [info     ] Currently computing chromosome /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr21.msp.tsv
2023-10-25 12:49:15 [info     ] Reading phenotypes and covariates in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip with shape (488266, 13)
First 5 lines of data 
                 PHENO  AGE  SEX  ...      PC8       PC9      PC10
id                                ...                             
1923142_1923142    0.0   41    0  ... -2.16198  1.098610  1.118510
5896937_5896937    0.0   53    0  ...  1.44203  1.102370  3.549290
2847894_2847894    NaN   58    1  ...  3.46699 -0.622341  1.860490
5764431_5764431    0.0   60    0  ... -1.80882 -0.505936  0.342399
4695056_4695056    0.0   65    0  ... -5.85230 -3.074070  1.981010

[5 rows x 13 columns]
2023-10-25 12:49:16 [info     ] Found mismatched pheno

admix.assoc.marginal: 100%|██████████| 1/1 [00:01<00:00,  1.54s/it]


2023-10-25 12:49:22 [info     ] Saving to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D.21.csv.gz
2023-10-25 12:49:28 [info     ] Currently computing chromosome /u/project/sgss/UKBB/UKB-ADMIXED/01-dataset/out/rfmix-hm3-lanc/chr22.msp.tsv
2023-10-25 12:49:29 [info     ] Reading phenotypes and covariates in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/test_comb.zip with shape (488266, 13)
First 5 lines of data 
                 PHENO  AGE  SEX  ...      PC8       PC9      PC10
id                                ...                             
1923142_1923142    0.0   41    0  ... -2.16198  1.098610  1.118510
5896937_5896937    0.0   53    0  ...  1.44203  1.102370  3.549290
2847894_2847894    NaN   58    1  ...  3.46699 -0.622341  1.860490
5764431_5764431    0.0   60    0  ... -1.80882 -0.505936  0.342399
4695056_4695056    0.0   65    0  ... -5.85230 -3.074070  1.981010

[5 rows x 13 columns]
2023-10-25 12:49:30 [info     ] Found mismatched pheno

admix.assoc.marginal: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


2023-10-25 12:49:34 [info     ] Saving to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D.22.csv.gz


In [11]:
os.system(f"python compileResult.py \
            -i /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D \
            -o /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc \
            -n T1D_250.1")




2023-10-25 12:52:54 [info     ] Currently compiling chromosomes in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D
2023-10-25 12:52:55 [info     ] Total of 27996 lanc segments
2023-10-25 12:52:56 [info     ] Saving association to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc.csv.gz and /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc.png
2023-10-25 12:52:58 [info     ] Saving to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc_qqplot.png and /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc_histo.png
2023-10-25 12:52:58 [info     ] Saving row of table to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/T1D_assoc_tableRow.npy


0

In [13]:
os.system(f"python tableCompile.py \
            -i /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/ \
            -o /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/table")

2023-10-25 12:53:43 [info     ] Currently compiling table in /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/
2023-10-25 12:53:43 [info     ] 1 found
2023-10-25 12:53:43 [info     ] Saving table to /u/home/z/zhuozshi/project-pasaniuc/ukbb_project/v2/test_folder/table.csv.gz


0